# Data Collection and Formatting

In [ ]:
import pandas as pd
import tarfile
from bs4 import BeautifulSoup
import requests
import re
import matplotlib.pyplot as plt
import seaborn as sns

%run -i "functions/unzip.py"
%run -i "functions/scrape_platforms.py"
%run -i "functions/scrape_SotU.py"
%run -i "functions/create_ts.py"
%run -i "functions/party_splits.py"

# Text Data
### Political Platforms

In [ ]:
years = ['1972', '1976', '1980', '1984', '1988', '1992', '1996', 
         '2000', '2008', '2012', '2016']
platforms = scrape_platforms(years)
platforms.head()

### State of the Union Addresses

In [ ]:
speeches = scrape_SotU()

### Debates

In [ ]:
url = 'https://www.presidency.ucsb.edu/documents/presidential-debate-the-university-nevada-las-vegas'
response = requests.get(url)
soup = BeautifulSoup(response.text, 'lxml')
debate = soup.find_all('p')
for deb in debate:
    print(deb.text)

# Survey Data
### Convert Data to Dataframes
#### Weights
Based on the sample weights provided by the General Social Survey, it appears that some samples were significantly over and underrepresented. In order to account for this weights will be applied to this analysis. In order to maintain a reasonable size of the data rather than achieving the right proportions by using the complete weights, I will use the rounded values to the nearest tenth.  

In [ ]:
weights = unzip('Weights')

In [ ]:
weights.head()

In [ ]:
weights['Weight variable'].describe()

In [ ]:
def weight(df):
    return df.loc[df.index.repeat(round(weights['Weight variable']*10, 0))].reset_index()

#### Voter Identity

In [ ]:
party = unzip('Voter_Identity')
party = party[['year', 'Political party affiliation']]
party = weight(party)
party.drop('index', axis = 1, inplace= True)

In [ ]:
party.head()

In [ ]:
party.rename(columns = {'Political party affiliation':'party'}, inplace=True)

In [ ]:
party['party'].value_counts()

In [ ]:
party.loc[party['party']=='Not str democrat', 'party']= 'Democrat'
party.loc[party['party']=='Not str republican', 'party']= 'Republican'
party.loc[party['party']=='Strong democrat', 'party']= 'Strong Democrat'
party.loc[party['party']=='Ind,near dem', 'party']= 'Lean Democrat'
party.loc[party['party']=='Strong republican', 'party']= 'Strong Republican'
party.loc[party['party']=='Ind,near rep', 'party']= 'Lean Republican'
party.loc[party['party']=='Other party', 'party']= 'Other'

parties = ['Democrat', 'Independent', 'Lean Democrat', 
           'Lean Republican', 'Other', 'Republican', 
           'Strong Democrat', 'Strong Republican']

percent_parties = create_ts(party, parties)
percent_parties.head()

In [ ]:
sns.lineplot(x= 'year', y = 'Lean Democrat', data= percent_parties)
sns.lineplot(x= 'year', y = 'Democrat', data= percent_parties)
sns.lineplot(x= 'year', y = 'Strong Democrat', data= percent_parties);

In [ ]:
sns.lineplot(x= 'year', y = 'Independent', data= percent_parties)
sns.lineplot(x= 'year', y = 'Other', data= percent_parties);

In [ ]:
sns.lineplot(x= 'year', y = 'Strong Republican', data= percent_parties)
sns.lineplot(x= 'year', y = 'Republican', data= percent_parties)
sns.lineplot(x= 'year', y = 'Lean Republican', data= percent_parties);

#### Abortion Opinions
To reduce dimensionality in the data, I will combine the reasons for abortion into three categories: health, economics, and right-to-choose. There will also be a category for abolishing legal abortion. 

In [ ]:
abortion = unzip('Abortion')
abortion.info()

In [ ]:
abortion.loc[(abortion['Strong chance of serious defect'] == 'Yes')|
             (abortion['Married--wants no more children'] == 'Yes')|
             (abortion["Woman's health seriously endangered"] == 'Yes')|
             (abortion["Low income--cant afford more children"] == 'Yes')|
             (abortion["Pregnant as result of rape"] == 'Yes')|
             (abortion['Not married']=='Yes'), 'abortion'] = 'conditional'

abortion.loc[(abortion['Strong chance of serious defect'] == 'Yes')&
             (abortion['Married--wants no more children'] == 'Yes')&
             (abortion["Woman's health seriously endangered"] == 'Yes')&
             (abortion["Low income--cant afford more children"] == 'Yes')&
             (abortion["Pregnant as result of rape"] == 'Yes')&
             (abortion['Not married']=='Yes'), 'abortion'] = 'unrestricted'

abortion.loc[(abortion['Strong chance of serious defect'] == 'No')&
             (abortion['Married--wants no more children'] == 'No')&
             (abortion["Woman's health seriously endangered"] == 'No')&
             (abortion["Low income--cant afford more children"] == 'No')&
             (abortion["Pregnant as result of rape"] == 'No')&
             (abortion['Not married']=='No'), 'abortion'] = 'never'

abortion.loc[abortion['Abortion if woman wants for any reason']== 'Yes', 'abortion']= 'unrestricted'
abortion.head()

In [ ]:
abortion = abortion[['year','id', 'abortion']]
abortion = weight(abortion)

In [ ]:
abortion[abortion['year']== 1986]

In [ ]:
abortion_percentages = create_ts(abortion, ['unrestricted','conditional','never'])
abortion_percentages.head()

In [ ]:
sns.lineplot(x= 'year', y= 'unrestricted', data= abortion_percentages, color = 'green')
sns.lineplot(x= 'year', y= 'conditional', data= abortion_percentages, color= 'yellow')
sns.lineplot(x= 'year', y= 'never', data= abortion_percentages, color= 'black');

In [ ]:
rep_ts_abort, dem_ts_abort, ind_ts_abort = party_splits(abortion, 
                                                        ['unrestricted', 
                                                         'conditional', 
                                                         'never'])

In [ ]:
fig = plt.figure(figsize=(20,5))
ax1, ax2, ax3 = fig.subplots(nrows=1, ncols=3, sharey= True)

sns.lineplot(x= 'year', y= 'unrestricted', data= rep_ts_abort, 
             color='red', label= 'Republican', ax = ax1)
sns.lineplot(x= 'year', y= 'unrestricted', data= dem_ts_abort, 
             color='blue', label= 'Democrat', ax = ax1)
sns.lineplot(x= 'year', y= 'unrestricted', data= ind_ts_abort, 
             color='purple', label= 'Independent', ax = ax1)
ax1.set_title('Unrestricted Access to Abortion by Party')
ax1.set_ylabel('Percent of Party')
ax1.set_xlabel('')

sns.lineplot(x= 'year', y= 'conditional', data= rep_ts_abort, 
             color='red', label= 'Republican', ax = ax2)
sns.lineplot(x= 'year', y= 'conditional', data= dem_ts_abort, 
             color='blue', label= 'Democrat', ax = ax2)
sns.lineplot(x= 'year', y= 'conditional', data= ind_ts_abort, 
             color='purple', label= 'Independent', ax = ax2)
ax2.set_title('Conditional Access to Abortion by Party')
ax2.set_xlabel('')

sns.lineplot(x= 'year', y= 'never', data= rep_ts_abort, 
             color='red', label= 'Republican', ax = ax3)
sns.lineplot(x= 'year', y= 'never', data= dem_ts_abort, 
             color='blue', label= 'Democrat', ax = ax3)
sns.lineplot(x= 'year', y= 'never', data= ind_ts_abort, 
             color='purple', label= 'Independent', ax = ax3)
ax3.set_title('Ban Access to Abortion by Party')
ax3.set_xlabel('')

plt.yticks(ticks=[.15, .3, .45, .6, .75], 
           labels=['','15%', '30%', '45%', '60%', '75%']);

It appears that access to abortion is increasingly a part of party identity. As we can see, the gap in beliefs about access to abortion has been increasing, with Democrats more likely to support unrestricted access, Republicans most likely to support conditions on access, and banning access altogether. 

In [ ]:
(str_rep, mid_rep, ln_rep, 
 str_dem, mid_dem, ln_dem, 
 oth, ind) = party_splits(abortion, ['unrestricted', 
                                     'conditional', 
                                     'never'], split = 'within party')

In [ ]:
fig = plt.figure(figsize=(20,5))
ax1, ax2, ax3 = fig.subplots(nrows=1, ncols=3, sharey= True)

sns.lineplot(x= 'year', y= 'unrestricted', data= str_rep, 
             color='red', label= 'Strong Republican', ax = ax1)
sns.lineplot(x= 'year', y= 'unrestricted', data= mid_rep, 
             color='blue', label= 'Republican', ax = ax1)
sns.lineplot(x= 'year', y= 'unrestricted', data= ln_rep, 
             color='purple', label= 'Lean Republican', ax = ax1)
ax1.set_title('Unrestricted Access to Abortion by Party')
ax1.set_ylabel('Percent of Party')
ax1.set_xlabel('')

sns.lineplot(x= 'year', y= 'conditional', data= str_rep, 
             color='red', label= 'Strong Republican', ax = ax2)
sns.lineplot(x= 'year', y= 'conditional', data= mid_rep, 
             color='blue', label= 'Republican', ax = ax2)
sns.lineplot(x= 'year', y= 'conditional', data= ln_rep, 
             color='purple', label= 'Lean Republican', ax = ax2)
ax2.set_title('Conditional Access to Abortion by Party')
ax2.set_xlabel('')

sns.lineplot(x= 'year', y= 'never', data= str_rep, 
             color='red', label= 'Strong Republican', ax = ax3)
sns.lineplot(x= 'year', y= 'never', data= mid_rep, 
             color='blue', label= 'Republican', ax = ax3)
sns.lineplot(x= 'year', y= 'never', data= ln_rep, 
             color='purple', label= 'Lean Republican', ax = ax3)
ax3.set_title('Ban Access to Abortion by Party')
ax3.set_xlabel('')

plt.yticks(ticks=[.15, .3, .45, .6, .75], 
           labels=['','15%', '30%', '45%', '60%', '75%']);

This trend is even more clear in these graphs, where we can see clearly that the more affiliated a person is the more likely that they support an outright ban. 

#### Family/ SES Background

In [ ]:
background = unzip('Background')

In [ ]:
background.columns

In [ ]:
for col in background.columns:
    print(background[col].value_counts())
    print('\n')

In [ ]:
background.rename(columns = {"Father's occupational prestige score using threshold method (2010)": 'father_prestige',
                             "Mother's occupational prestige score using threshold method (2010)": 'mother_prestige',
                             'Religion in which raised': 'raised_relig', 
                             'How many grandparents born outside u.s.': 'immigrant_gpar', 
                             'Were rs parents born in this country': 'immigrant_par', 
                             'Was r born in this country': 'immigrant'}, inplace=True)

I will encode the immigration status of parents so that the number of grandparents and parents born outside of the country are counted. Parents born in the U.S will be coded as 0s. Because this data will likely not be used for modeling, I will keep not applicable values as they are. 

In [ ]:
background.loc[(background['raised_relig']=='Not applicable')|
               (background['raised_relig']=='No answer')|
               (background['raised_relig']=="Don't know"), 'raised_relig']= 'Other'
background.loc[(background['immigrant_gpar']== 'All in u.s'), 'immigrant_gpar']= 0
background.loc[(background['immigrant_par']=='Both in u.s')|
               (background['immigrant_par']=='Mother; fa. dk')|
               (background['immigrant_par']=='Father; mo.dk'), 'immigrant_par']= 0
background.loc[(background['immigrant_par']=='Neither in u.s')|
               (background['immigrant_par']=='Not mother;fa.dk')|
               (background['immigrant_par']=='Not father;mo.dk'), 'immigrant_par']= 2
background.loc[(background['immigrant_par']=='Mother only')|
               (background['immigrant_par']=='Father only'), 'immigrant_par']= 1
background.loc[(background['immigrant']=='Yes'), 'immigrant']= 'no'
background.loc[(background['immigrant']=='No'), 'immigrant']= 'yes'
background.loc[(background['immigrant']=='Not applicable')|
               (background['immigrant']=='No answer')|
               (background['immigrant']=="Don't know"), 'immigrant']= 'n/a'

The 'Rs living standard compared to parents' is messy and highly incomplete, so I will drop it for this analysis. In addition, after consideration, parental education levels are more appropriately placed in the education table so they will also be dropped from this table.  

In [ ]:
background.drop(['Rs living standard compared to parents', 
                 'Fathers highest degree', 'Mothers highest degree'], axis= 1, inplace= True)

In [ ]:
background = weight(background)

#### Criminal Justice Attitudes

In [ ]:
crim_justice = unzip('Criminal_Justice')
crim_justice.columns

In [ ]:
crim_justice[crim_justice['Ever approve of police striking citizen']=='Yes']['Citizen attacking policeman with fists'].value_counts()

The police striking citizens columns lack information on how the police response is defined, and will likely be difficult to interpret given the nature of this analysis. Unfortunately I will have to drop them. 

In [ ]:
crim_justice.drop(['Citizen attacking policeman with fists', 
                   'Citizen attempting to escape custody', 
                   'Citizen questioned as murder suspect', 
                   'Citizen said vulgar or obscene things', 
                   'Ever approve of police striking citizen'], axis = 1, inplace = True)

In [ ]:
crim_justice.rename(columns = {'Should marijuana be made legal': 'legalize_marijuana',
                             'Courts dealing with criminals': 'sentencing',
                             'Favor or oppose death penalty for murder': 'death_penalty', 
                             'Afraid to walk at night in neighborhood': 'fear'}, inplace=True)

In [ ]:
crim_justice = weight(crim_justice)

In [ ]:
crim_justice['legalize_marijuana'].unique()

In [ ]:
rep_ts_mj, dem_ts_mj, ind_ts_mj = party_splits(crim_justice, 
                                               ['Legal',
                                                'Not legal'])

In [ ]:
fig = plt.figure(figsize=(20,5))
ax1 = fig.subplots(nrows=1, ncols=1, sharey= True)

sns.lineplot(x= 'year', y= 'Legal', data= rep_ts_mj, 
             color='red', label= 'Republican', ax = ax1)
sns.lineplot(x= 'year', y= 'Legal', data= dem_ts_mj, 
             color='blue', label= 'Democrat', ax = ax1)
sns.lineplot(x= 'year', y= 'Legal', data= ind_ts_mj, 
             color='purple', label= 'Independent', ax = ax1)
ax1.set_title('Legalization of Marijuana by Party')
ax1.set_ylabel('Percent of Party')
ax1.set_xlabel('')

plt.yticks(ticks=[.15, .3, .45, .6, .75], 
           labels=['','15%', '30%', '45%', '60%', '75%']);

Clearly, attitudes about the legalization of marijuana have changed in favor of legalization, with Democrats leading the way. Another example of a widening partisan divide around certain issues. 

In [ ]:
(str_rep, mid_rep, ln_rep, 
 str_dem, mid_dem, ln_dem, 
 oth, ind) = party_splits(crim_justice, ['Legal', 
                                     'Not legal'], split = 'within party')

In [ ]:
fig = plt.figure(figsize=(20,5))
ax1, ax2, ax3 = fig.subplots(nrows=1, ncols=3, sharey= True)

sns.lineplot(x= 'year', y= 'Legal', data= str_rep, 
             color='red', label= 'Strong Republican', ax = ax1)
sns.lineplot(x= 'year', y= 'Legal', data= mid_rep, 
             color='blue', label= 'Republican', ax = ax1)
sns.lineplot(x= 'year', y= 'Legal', data= ln_rep, 
             color='purple', label= 'Lean Republican', ax = ax1)
ax1.set_title('Legalization of Marijuana by Republican Affiliation')
ax1.set_ylabel('Percent of Party')
ax1.set_xlabel('')

sns.lineplot(x= 'year', y= 'Legal', data= str_dem, 
             color='red', label= 'Strong Democrat', ax = ax2)
sns.lineplot(x= 'year', y= 'Legal', data= mid_dem, 
             color='blue', label= 'Democratic', ax = ax2)
sns.lineplot(x= 'year', y= 'Legal', data= ln_dem, 
             color='purple', label= 'Lean Democratic', ax = ax2)
ax2.set_title('Conditional Access to Abortion by Party')
ax2.set_xlabel('')

sns.lineplot(x= 'year', y= 'Legal', data= oth, 
             color='red', label= 'Other', ax = ax3)
sns.lineplot(x= 'year', y= 'Legal', data= ind, 
             color='blue', label= 'Independent', ax = ax3)
ax3.set_title('Ban Access to Abortion by Party')
ax3.set_xlabel('')

plt.yticks(ticks=[.15, .3, .45, .6, .75], 
           labels=['','15%', '30%', '45%', '60%', '75%']);

In [ ]:
crim_justice['sentencing'].unique()

In [ ]:
rep_ts_sen, dem_ts_sen, ind_ts_sen = party_splits(crim_justice, 
                                               ['About right',
                                                'Not harsh enough', 
                                                'Too harsh'])

In [ ]:
fig = plt.figure(figsize=(20,5))
ax1, ax2, ax3 = fig.subplots(nrows=1, ncols=3, sharey= True)

sns.lineplot(x= 'year', y= 'About right', data= rep_ts_sen, 
             color='red', label= 'Republican', ax = ax1)
sns.lineplot(x= 'year', y= 'About right', data= dem_ts_sen, 
             color='blue', label= 'Democrat', ax = ax1)
sns.lineplot(x= 'year', y= 'About right', data= ind_ts_sen, 
             color='purple', label= 'Independent', ax = ax1)
ax1.set_title('Criminal Sentencing is about right')
ax1.set_ylabel('Percent of Party')
ax1.set_xlabel('')

sns.lineplot(x= 'year', y= 'Not harsh enough', data= rep_ts_sen, 
             color='red', label= 'Republican', ax = ax2)
sns.lineplot(x= 'year', y= 'Not harsh enough', data= dem_ts_sen, 
             color='blue', label= 'Democrat', ax = ax2)
sns.lineplot(x= 'year', y= 'Not harsh enough', data= ind_ts_sen, 
             color='purple', label= 'Independent', ax = ax2)
ax2.set_title('Criminal Sentencing is not harsh enough')
ax2.set_xlabel('')

sns.lineplot(x= 'year', y= 'Too harsh', data= rep_ts_sen, 
             color='red', label= 'Republican', ax = ax3)
sns.lineplot(x= 'year', y= 'Too harsh', data= dem_ts_sen, 
             color='blue', label= 'Democrat', ax = ax3)
sns.lineplot(x= 'year', y= 'Too harsh', data= ind_ts_sen, 
             color='purple', label= 'Independent', ax = ax3)
ax3.set_title('Criminal Sentencing is too harsh')
ax3.set_xlabel('')

plt.yticks(ticks=[.15, .3, .45, .6, .75], 
           labels=['','15%', '30%', '45%', '60%', '75%']);

It seems as though the people who think criminal sentencing is about right are not clearly from one party or another. When it comes to sentencing otherwise a more clear difference emerges. 

In [ ]:
(str_rep, mid_rep, ln_rep, 
 str_dem, mid_dem, ln_dem, 
 oth, ind) = party_splits(crim_justice, ['About right',
                                         'Not harsh enough', 
                                         'Too harsh'], split='within party')

In [ ]:
fig = plt.figure(figsize=(20,5))
ax1, ax2, ax3 = fig.subplots(nrows=1, ncols=3, sharey= True)

sns.lineplot(x= 'year', y= 'About right', data= str_rep, 
             color='red', label= 'Republican', ax = ax1)
sns.lineplot(x= 'year', y= 'About right', data= mid_rep, 
             color='blue', label= 'Democrat', ax = ax1)
sns.lineplot(x= 'year', y= 'About right', data= ln_rep, 
             color='purple', label= 'Independent', ax = ax1)
ax1.set_title('Criminal Sentencing is about right')
ax1.set_ylabel('Percent of Party')
ax1.set_xlabel('')

sns.lineplot(x= 'year', y= 'Not harsh enough', data= str_rep, 
             color='red', label= 'Republican', ax = ax2)
sns.lineplot(x= 'year', y= 'Not harsh enough', data= mid_rep, 
             color='blue', label= 'Democrat', ax = ax2)
sns.lineplot(x= 'year', y= 'Not harsh enough', data= ln_rep, 
             color='purple', label= 'Independent', ax = ax2)
ax2.set_title('Criminal Sentencing is not harsh enough')
ax2.set_xlabel('')

sns.lineplot(x= 'year', y= 'Too harsh', data= str_rep, 
             color='red', label= 'Republican', ax = ax3)
sns.lineplot(x= 'year', y= 'Too harsh', data= mid_rep, 
             color='blue', label= 'Democrat', ax = ax3)
sns.lineplot(x= 'year', y= 'Too harsh', data= ln_rep, 
             color='purple', label= 'Independent', ax = ax3)
ax3.set_title('Criminal Sentencing is too harsh')
ax3.set_xlabel('')

plt.yticks(ticks=[.15, .3, .45, .6, .75], 
           labels=['','15%', '30%', '45%', '60%', '75%']);

##### Education

In [ ]:
education = unzip('Education')
education.columns

In [ ]:
for col in education.columns:
    print(education[col].value_counts())
    print('\n')

With so many not applicable entries for the field of degree, it is more appropriate to remove them from the analysis. 

In [ ]:
education.drop(['The field of degree r earned'], axis=1, inplace=True)

In [ ]:
education.rename(columns = {'Highest year of school completed':'school_years',
                            'Highest year school completed, father':'school_years_dad', 
                            'Highest year school completed, mother':'school_years_mom',
                            'Rs highest degree':'degree',
                            'Fathers highest degree':'degree_dad',
                            'Mothers highest degree': 'degree_mom'}, inplace=True)

Encode nan information as a placeholder so that it can be easily removed later in the analysis. 

In [ ]:
education.loc[(education['school_years']=='No answer')|
              (education['school_years']=='Not applicable')|
              (education['school_years']== "Don't know"), 'school_years']= -9
education.loc[(education['school_years_dad']=='No answer')|
              (education['school_years_dad']=='Not applicable')|
              (education['school_years_dad']== "Don't know"), 'school_years_dad']= -9
education.loc[(education['school_years_mom']=='No answer')|
              (education['school_years_mom']=='Not applicable')|
              (education['school_years_mom']== "Don't know"), 'school_years_mom']= -9

In [ ]:
education.head()

In [ ]:
education = weight(education)

##### Employment

In [ ]:
employment = unzip('Employment')
employment.columns

In [ ]:
for col in employment.columns:
    print(employment[col].value_counts())
    print('\n')

In [ ]:
employment.drop(['Is r likely to lose job', 'Could r find equally good job', 
                 'Job or housework', 'If rich, continue or stop working', 
                 'Workers need strong unions'], axis=1, inplace = True)

In [ ]:
employment.rename(columns= {'Does r or spouse belong to union': 'union',
                            'Number of hours usually work a week': 'wrk_hrs',
                            'Labor force status': 'job_status',
                            'Rs occupational prestige score using threshold method (2010)': 'prestige'}, 
                  inplace = True)

In [ ]:
employment.loc[(employment['union']=='Neither belongs', 'union')]= 'non-member'
employment.loc[(employment['union']=='R belongs')|
               (employment['union']=='Spouse belongs')|
               (employment['union']=='R and spouse belong'), 'union']= 'member'

In [ ]:
employment.head()

In [ ]:
employment = weight(employment)

###### Family Values

In [ ]:
fam_vals = unzip('Family_Values')
fam_vals.columns

In [ ]:
for col in fam_vals.columns:
    print(fam_vals[col].value_counts())
    print('\n')

In [ ]:
fam_vals.rename(columns= {'Better for man to work, woman tend home': 'trad_roles', 
                          'Preschool kids suffer if mother works': 'mom_wrk_kids_suffer', 
                          'Ideal number of children': 'bst_num_chld', 
                          'To help others': 'tch_to_help',
                          'To work hard': 'tch_to_wrk', 
                          'To think for ones self': 'tch_ind', 
                          'To be well liked or popular': 'tch_pop',
                          'To obey': 'tch_obey', 
                          'Number of children': 'num_chld',
                          'Number of family generations in household': 'num_gen'}, inplace=True)

In [ ]:
fam_vals.drop(['Rs kids living standard compared to r', 'Rs living standard compared to parents', 
               'Highest year school completed, mother', 'Highest year school completed, father', 
               'Number of brothers and sisters', 'Favor spanking to discipline child'], axis=1, inplace=True)

In [ ]:
fam_vals.loc[fam_vals['num_gen']=='1 gen', 'num_gen']= 1
fam_vals.loc[(fam_vals['num_gen']=='2 gens, children')|
             (fam_vals['num_gen']=='2 gens, parents')|
             (fam_vals['num_gen']=='2 gens, grandchldrn'), 'num_gen']= 2
fam_vals.loc[(fam_vals['num_gen']=='3 gens, grandchldrn')|
             (fam_vals['num_gen']=='3 gens, chld, par'), 'num_gen']= 3
fam_vals.loc[fam_vals['num_gen']=='4 gens', 'num_gen']= 4

In [ ]:
fam_vals.head()

In [ ]:
fam_vals = weight(fam_vals)

###### Gender and Sexuality

In [ ]:
gender = unzip('Gender_and_Sexuality')
gender.columns

In [ ]:
for col in gender.columns:
    print(gender[col].value_counts())
    print('\n')

In [ ]:
gender.drop(['Should hire and promote women                            ', 
             'For or against preferential hiring of women      ', 
             'Better for man to work, woman tend home', 'Preschool kids suffer if mother works',
             'Sex before marriage -- teens 14-16', 'Birth control to teenagers 14-16'], 
            axis = 1, inplace = True)

In [ ]:
gender.rename(columns ={'Sex before marriage':'sex_before_marr',
                        'Divorce laws': 'divorce', 
                        'Sex education in public schools':'sex_ed',
                        'Women not suited for politics': 'no_women_pol',
                        'Homosexuals should have right to marry':'gay_marr'}, inplace=True)

In [ ]:
gender = weight(gender)

In [ ]:
gender.loc[(gender['gay_marr']=='Neither agree nor disagree')|
           (gender['gay_marr']=='Cant choose'), 'gay_marr']= 'undecided'

In [ ]:
gender['gay_marr'].unique()

In [ ]:
(rep_ts_gay, dem_ts_gay, ind_ts_gay) = party_splits(gender[['year', 'gay_marr']], ['Strongly disagree', 
                                         'undecided',
                                         'Disagree',
                                         'Agree', 
                                         'Strongly agree'])

In [ ]:
fig = plt.figure(figsize=(20,5))
ax1, ax2, ax3, ax4, ax5 = fig.subplots(nrows=1, ncols=5, sharey= True)

sns.lineplot(x= 'year', y= 'Strongly disagree', data= rep_ts_gay, 
             color='red', label= 'Republican', ax = ax1)
sns.lineplot(x= 'year', y= 'Strongly disagree', data= dem_ts_gay, 
             color='blue', label= 'Democrat', ax = ax1)
sns.lineplot(x= 'year', y= 'Strongly disagree', data= ind_ts_gay, 
             color='purple', label= 'Independent', ax = ax1)
ax1.set_title('Strongly Disagree with Right to Marry by Party')
ax1.set_ylabel('Percent of Party')
ax1.set_xlabel('')

sns.lineplot(x= 'year', y= 'Disagree', data= rep_ts_gay, 
             color='red', label= 'Republican', ax = ax2)
sns.lineplot(x= 'year', y= 'Disagree', data= dem_ts_gay, 
             color='blue', label= 'Democrat', ax = ax2)
sns.lineplot(x= 'year', y= 'Disagree', data= ind_ts_gay, 
             color='purple', label= 'Independent', ax = ax2)
ax2.set_title('Disagree with Right to Marry by Party')
ax2.set_xlabel('')

sns.lineplot(x= 'year', y= 'undecided', data= rep_ts_gay, 
             color='red', label= 'Republican', ax = ax3)
sns.lineplot(x= 'year', y= 'undecided', data= dem_ts_gay, 
             color='blue', label= 'Democrat', ax = ax3)
sns.lineplot(x= 'year', y= 'undecided', data= ind_ts_gay, 
             color='purple', label= 'Independent', ax = ax3)
ax3.set_title('Undecided about right to Marry by Party')
ax3.set_xlabel('')

sns.lineplot(x= 'year', y= 'Agree', data= rep_ts_gay, 
             color='red', label= 'Republican', ax = ax4)
sns.lineplot(x= 'year', y= 'Agree', data= dem_ts_gay, 
             color='blue', label= 'Democrat', ax = ax4)
sns.lineplot(x= 'year', y= 'Agree', data= ind_ts_gay, 
             color='purple', label= 'Independent', ax = ax4)
ax4.set_title('Agree with right to Marry by Party')
ax4.set_xlabel('')

sns.lineplot(x= 'year', y= 'Strongly agree', data= rep_ts_gay, 
             color='red', label= 'Republican', ax = ax5)
sns.lineplot(x= 'year', y= 'Strongly agree', data= dem_ts_gay, 
             color='blue', label= 'Democrat', ax = ax5)
sns.lineplot(x= 'year', y= 'Strongly agree', data= ind_ts_gay, 
             color='purple', label= 'Independent', ax = ax5)
ax5.set_title('Strongly Agree with right to Marry by Party')
ax5.set_xlabel('')

plt.yticks(ticks=[.15, .3, .45, .6, .75], 
           labels=['','15%', '30%', '45%', '60%', '75%']);

In [ ]:
(str_rep, mid_rep, ln_rep, 
 str_dem, mid_dem, ln_dem, 
 oth, ind) = party_splits(gender[['year','gay_marr']], ['Strongly disagree', 
                                         'undecided',
                                         'Disagree',
                                         'Agree', 
                                         'Strongly agree'], split='within party')

In [ ]:
fig = plt.figure(figsize=(20,5))
ax1, ax2, ax3, ax4, ax5 = fig.subplots(nrows=1, ncols=5, sharey= True)

sns.lineplot(x= 'year', y= 'Strongly disagree', data= str_rep, 
             color='red', label= 'Republican', ax = ax1)
sns.lineplot(x= 'year', y= 'Strongly disagree', data= mid_rep, 
             color='blue', label= 'Democrat', ax = ax1)
sns.lineplot(x= 'year', y= 'Strongly disagree', data= ln_rep, 
             color='purple', label= 'Independent', ax = ax1)
ax1.set_title('Strongly Disagree with Right to Marry by Party')
ax1.set_ylabel('Percent of Party')
ax1.set_xlabel('')

sns.lineplot(x= 'year', y= 'Disagree', data= str_rep, 
             color='red', label= 'Republican', ax = ax2)
sns.lineplot(x= 'year', y= 'Disagree', data= mid_rep, 
             color='blue', label= 'Democrat', ax = ax2)
sns.lineplot(x= 'year', y= 'Disagree', data= ln_rep, 
             color='purple', label= 'Independent', ax = ax2)
ax2.set_title('Disagree with Right to Marry by Party')
ax2.set_xlabel('')

sns.lineplot(x= 'year', y= 'undecided', data= str_rep, 
             color='red', label= 'Republican', ax = ax3)
sns.lineplot(x= 'year', y= 'undecided', data= mid_rep, 
             color='blue', label= 'Democrat', ax = ax3)
sns.lineplot(x= 'year', y= 'undecided', data= ln_rep, 
             color='purple', label= 'Independent', ax = ax3)
ax3.set_title('Undecided about right to Marry by Party')
ax3.set_xlabel('')

sns.lineplot(x= 'year', y= 'Agree', data= str_rep, 
             color='red', label= 'Republican', ax = ax4)
sns.lineplot(x= 'year', y= 'Agree', data= mid_rep, 
             color='blue', label= 'Democrat', ax = ax4)
sns.lineplot(x= 'year', y= 'Agree', data= ln_rep, 
             color='purple', label= 'Independent', ax = ax4)
ax4.set_title('Agree with right to Marry by Party')
ax4.set_xlabel('')

sns.lineplot(x= 'year', y= 'Strongly agree', data= str_rep, 
             color='red', label= 'Republican', ax = ax5)
sns.lineplot(x= 'year', y= 'Strongly agree', data= mid_rep, 
             color='blue', label= 'Democrat', ax = ax5)
sns.lineplot(x= 'year', y= 'Strongly agree', data= ln_rep, 
             color='purple', label= 'Independent', ax = ax5)
ax5.set_title('Strongly Agree with right to Marry by Party')
ax5.set_xlabel('')

plt.yticks(ticks=[.15, .3, .45, .6, .75], 
           labels=['','15%', '30%', '45%', '60%', '75%']);

###### Identity

In [ ]:
pers_id = unzip('Identity')
pers_id.columns

In [ ]:
for col in pers_id.columns:
    print(pers_id[col].value_counts())
    print('\n')

In [ ]:
pers_id.rename(columns={'Age of respondent':'age', 
                      'Respondents sex':'sex', 
                      'Race of respondent':'race',
                      'Region of interview':'region'}, inplace = True)

In [ ]:
pers_id = weight(pers_id)

###### Lifestyle

In [ ]:
lifestyle = unzip('Lifestyle')
lifestyle.columns

In [ ]:
for col in lifestyle.columns:
    print(lifestyle[col].value_counts())
    print('\n')

In [ ]:
lifestyle.rename(columns = {'Hours per day watching tv':'hrs_tv',
                            'How often does r read newspaper':'paper',
                            'Have gun in home':'gun',
                            'Is life exciting or dull':'life',
                            'General happiness':'happy',
                            "R's age when 1st child born":'age_at_frst_chld',
                            'Marital status':'married',
                            'Does r own or rent home?':'rent_or_own'}, inplace = True)

In [ ]:
lifestyle.drop(['Does r or spouse hunt', 'Spend evening at bar', 
                'Spend evening with friends', 'Spend evening with neighbor', 
                'Spend evening with relatives', 'Age of respondent', 'Number of children',
                'Number of hours usually work a week'], axis=1, inplace=True)

In [ ]:
lifestyle = weight(lifestyle)

###### Opinions

In [ ]:
opinions = unzip('Opinions')
opinions.columns

In [ ]:
for col in opinions.columns:
    print(opinions[col].value_counts())
    print('\n')

In [ ]:
opinions = weight(opinions)

###### Race Relations

In [ ]:
race = unzip('Race_Relations')
race.columns

In [ ]:
for col in race.columns:
    print(race[col].value_counts())
    print('\n')

Because of the ways in which these variables were coded, many of them are either redundant with other questions, or difficult to gain meaningful information from.  

In [ ]:
race.drop(['Hard working - lazy', 'Hard working - lazy.1', 
           'Rich - poor', 'Rich - poor.1', 'Blacks overcome prejudice without favors ',
           'Any opp. race in neighborhood', 'Favor law against racial intermarriage', 
           'Improving the conditions of blacks'], axis = 1, inplace = True)

In [ ]:
race.loc[race['Should govt aid blacks?']=='Agree with both', 'Should govt aid blacks?']= 'no opinion'
race.loc[race['Should govt aid blacks?']=='No special treatment', 'Should govt aid blacks?']= 'strongly oppose'
race.loc[race['Should govt aid blacks?']== 4, 'Should govt aid blacks?']= 'oppose'
race.loc[race['Should govt aid blacks?']=='Govt help blks', 'Should govt aid blacks?']= 'strongly favor'
race.loc[race['Should govt aid blacks?']== 2, 'Should govt aid blacks?']= 'favor'
race.loc[race['Should govt aid blacks?']=="Don't know", 'Should govt aid blacks?']= 'no opinion'
race.loc[race['Should govt aid blacks?']=="No answer", 'Should govt aid blacks?']= 'no opinion'

In [ ]:
race = weight(race)

###### Religion

In [ ]:
religion = unzip('Religion')
religion.columns

In [ ]:
for col in religion.columns:
    print(religion[col].value_counts())
    print('\n')

In [ ]:
religion.drop(['Bible prayer in public schools'], axis = 1, inplace = True)

In [ ]:
religion = weight(religion)

###### Socioeconomic Status

In [ ]:
ses = unzip('SES')
ses.columns

In [ ]:
for col in ses.columns:
    print(ses[col].value_counts())
    print('\n')

In [ ]:
ses.drop(['Rs income in constant $', 'Standard of living of r will improve', 
          'Rs kids living standard compared to r', 'Rs living standard compared to parents', 
          'Change in financial situation', 'Rs occupational prestige score using threshold method (2010)'], 
         axis = 1, inplace = True)

In [ ]:
ses = weight(ses)

###### Systems

In [ ]:
systems = unzip('Systems')
systems.columns

In [ ]:
for col in systems.columns:
    print(systems[col].value_counts())
    print('\n')

In [ ]:
systems.drop(['Confidence in medicine'], axis = 1, inplace = True)

In [ ]:
systems = weight(systems)